In [2]:
!pip install torch torchvision tensorflow pillow matplotlib numpy



In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Force CPU-only mode

In [2]:
import torch
import torch.nn.functional as F
import tensorflow as tf
import numpy as np
from PIL import Image
from torchvision import transforms
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
from IPython.display import display

2025-05-03 20:33:35.796748: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746293615.831484    5635 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746293615.841070    5635 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746293615.867272    5635 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746293615.867360    5635 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746293615.867363    5635 computation_placer.cc:177] computation placer alr

In [2]:
import os
import tensorflow as tf
import torch
import numpy as np
from PIL import Image

# Configure GPU settings at the very beginning
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'  # Allow GPU memory growth

# Initialize TensorFlow in a separate cell before any model loading
try:
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
        except RuntimeError as e:
            print("GPU configuration error (memory growth):", e)
except Exception as e:
    print("GPU detection error:", e)


In [70]:
class SkinDiseaseClassifier:
    def __init__(self):
        print("Initializing models...")
        
        # Initialize TensorFlow model first
        try:
            print("Loading TensorFlow model...")
            self.model1 = tf.keras.models.load_model('final_model.h5')
            self.feature_extractor = tf.keras.applications.ResNet50(
                weights='imagenet', 
                include_top=False, 
                pooling='avg'
            )
            print("TensorFlow model loaded successfully")
        except Exception as e:
            print("Error loading TensorFlow model:", e)
            raise

        # Then initialize PyTorch model
        try:
            print("Loading PyTorch model...")
            self.model2 = torch.jit.load('model_mobile_quantized.pt')
            self.model2.eval()
            print("PyTorch model loaded successfully")
        except Exception as e:
            print("Error loading PyTorch model:", e)
            raise

        # Class names
        self.class_names_model1 = [
            'Eczema', 'Melanoma', 'Atopic Dermatitis', 
            'Basal Cell Carcinoma', 'Melanocytic Nevi', 'Benign Keratosis'
        ]
        
        self.class_names_model2 = [
            'Basal Cell Carcinoma', 'Darier_s Disease', 'Epidermolysis Bullosa Pruriginosa',
            'Hailey-Hailey Disease', 'Herpes Simplex', 'Impetigo', 'Larva Migrans',
            'Leprosy Borderline', 'Leprosy Lepromatous', 'Leprosy Tuberculoid',
            'Lichen Planus', 'Lupus Erythematosus Chronicus Discoides', 'Melanoma',
            'Molluscum Contagiosum', 'Mycosis Fungoides', 'Neurofibromatosis',
            'Papilomatosis Confluentes And Reticulate', 'Pediculosis Capitis',
            'Pityriasis Rosea', 'Porokeratosis Actinic', 'Psoriasis', 'Tinea Corporis',
            'Tinea Nigra', 'Tungiasis', 'actinic keratosis', 'dermatofibroma', 'nevus',
            'pigmented benign keratosis', 'seborrheic keratosis', 'squamous cell carcinoma',
            'vascular lesion'
        ]

    def preprocess_for_model1(self, image):
        """Preprocess image for TensorFlow ResNet50 model"""
        image = image.resize((224, 224))
        image_array = img_to_array(image)
        image_array = np.expand_dims(image_array, axis=0)
        return preprocess_input(image_array)

    def preprocess_for_model2(self, image):
        """Preprocess image for PyTorch DINOv2 model"""
        preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        return preprocess(image.convert("RGB")).unsqueeze(0)

    def predict(self, image_path):
        """Run parallel prediction on both models"""
        # Load image
        pil_image = Image.open(image_path)
        
        # Run model1 (TensorFlow)
        try:
            tf_input = self.preprocess_for_model1(pil_image)
            features = self.feature_extractor.predict(tf_input)
            model1_probs = self.model1.predict(features)[0]
            model1_class_idx = np.argmax(model1_probs)
            model1_confidence = model1_probs[model1_class_idx]
            model1_class = self.class_names_model1[model1_class_idx]
        except Exception as e:
            print("TensorFlow prediction error:", e)
            model1_class = "Error"
            model1_confidence = 0.0
            model1_probs = []
        
        # Run model2 (PyTorch)
        try:
            pt_input = self.preprocess_for_model2(pil_image)
            with torch.no_grad():
                model2_output = self.model2(pt_input)
            model2_probs = F.softmax(model2_output, dim=-1)[0]
            model2_class_idx = model2_probs.argmax(-1).item()
            model2_confidence = model2_probs[model2_class_idx].item()
            model2_class = self.class_names_model2[model2_class_idx]
        except Exception as e:
            print("PyTorch prediction error:", e)
            model2_class = "Error"
            model2_confidence = 0.0
            model2_probs = []
        
        return {
            'model1': {
                'class': model1_class,
                'confidence': float(model1_confidence),
                'all_probs': model1_probs.tolist() if len(model1_probs) > 0 else []
            },
            'model2': {
                'class': model2_class,
                'confidence': model2_confidence,
                'all_probs': model2_probs.tolist() if torch.is_tensor(model2_probs) else []
            },
            'combined': self.combine_results(
                model1_class, model1_confidence,
                model2_class, model2_confidence
            )
        }

    def combine_results(self, class1, conf1, class2, conf2):
        """Class-aware combination logic"""
        # Known conflict cases
        conflict_pairs = {
            ('Eczema', 'Psoriasis'): 'Psoriasis',
            ('Benign Keratosis', 'actinic keratosis'): 'actinic keratosis',
            ('Melanocytic Nevi', 'Melanoma'): 'Melanoma',
            ('Atopic Dermatitis', 'Eczema'): 'Eczema'
        }
        
        # Check both orderings of class pairs
        if (class1, class2) in conflict_pairs:
            correct_class = conflict_pairs[(class1, class2)]
            return {
                'final_class': correct_class, 
                'confidence': max(conf1, conf2),
                'source': 'expert_rules'
            }
        elif (class2, class1) in conflict_pairs:
            correct_class = conflict_pairs[(class2, class1)]
            return {
                'final_class': correct_class,
                'confidence': max(conf1, conf2),
                'source': 'expert_rules'
            }
        
        # Default confidence-based approach with preference to model2 (DINOv2)
        if conf1 > conf2 + 0.15:  # If model1 is significantly more confident
            return {
                'final_class': class1,
                'confidence': conf1,
                'source': 'model1'
            }
        else:  # Default to model2's prediction
            return {
                'final_class': class2,
                'confidence': conf2,
                'source': 'model2'
            }

In [71]:
classifier = SkinDiseaseClassifier()


Initializing models...
Loading TensorFlow model...
TensorFlow model loaded successfully
Loading PyTorch model...
PyTorch model loaded successfully


In [76]:
# Test prediction
image_path = '/home/rabieash/projects/GP/Kareem_eczema.jpeg'
result = classifier.predict(image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


In [77]:
# Display results
print("\n=== Prediction Results ===")
print("ResNet50 Prediction:", result['model1']['class'], f"({result['model1']['confidence']:.1%})")
print("DINOv2 Prediction:", result['model2']['class'], f"({result['model2']['confidence']:.1%})")
print("Final Decision:", result['combined']['final_class'], 
      f"({result['combined']['confidence']:.1%})",
      f"[Source: {result['combined']['source']}]")


=== Prediction Results ===
ResNet50 Prediction: Atopic Dermatitis (67.1%)
DINOv2 Prediction: Impetigo (48.3%)
Final Decision: Atopic Dermatitis (67.1%) [Source: model1]


**l7ad hna**---------------------------------------------------------------------------------------

In [79]:
!pip install joblib

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)


In [2]:
import os
import dill
import torch
import torch.nn.functional as F
import tensorflow as tf
import numpy as np
from PIL import Image
from torchvision import transforms
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from pathlib import Path

class CombinedSkinModel:
    def __init__(self, tf_model_path, torch_model_path, class_names1, class_names2, conflict_rules):
        # Store paths instead of loaded models
        self.tf_model_path = tf_model_path
        self.torch_model_path = torch_model_path
        self.class_names_model1 = class_names1
        self.class_names_model2 = class_names2
        self.conflict_rules = conflict_rules
        
        # Initialize models as None (will lazy-load when needed)
        self._tf_model = None
        self._torch_model = None
        self._feature_extractor = None
    
    @property
    def tf_model(self):
        if self._tf_model is None:
            self._tf_model = tf.keras.models.load_model(self.tf_model_path)
        return self._tf_model
    
    @property
    def torch_model(self):
        if self._torch_model is None:
            self._torch_model = torch.jit.load(self.torch_model_path)
            self._torch_model.eval()
        return self._torch_model
    
    @property
    def feature_extractor(self):
        if self._feature_extractor is None:
            self._feature_extractor = tf.keras.applications.ResNet50(
                weights='imagenet', 
                include_top=False, 
                pooling='avg'
            )
        return self._feature_extractor
    
    def preprocess_for_model1(self, image):
        """Preprocess image for TensorFlow ResNet50 model"""
        image = image.resize((224, 224))
        image_array = img_to_array(image)
        image_array = np.expand_dims(image_array, axis=0)
        return preprocess_input(image_array)

    def preprocess_for_model2(self, image):
        """Preprocess image for PyTorch DINOv2 model"""
        preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        return preprocess(image.convert("RGB")).unsqueeze(0)
    
    def combine_results(self, class1, conf1, class2, conf2):
        """Class-aware combination logic"""
        # Check both orderings of class pairs
        if (class1, class2) in self.conflict_rules:
            correct_class = self.conflict_rules[(class1, class2)]
            return {
                'final_class': correct_class, 
                'confidence': max(conf1, conf2),
                'source': 'expert_rules'
            }
        elif (class2, class1) in self.conflict_rules:
            correct_class = self.conflict_rules[(class2, class1)]
            return {
                'final_class': correct_class,
                'confidence': max(conf1, conf2),
                'source': 'expert_rules'
            }
        
        # Default confidence-based approach with preference to model2 (DINOv2)
        if conf1 > conf2 + 0.15:  # If model1 is significantly more confident
            return {
                'final_class': class1,
                'confidence': conf1,
                'source': 'model1'
            }
        else:  # Default to model2's prediction
            return {
                'final_class': class2,
                'confidence': conf2,
                'source': 'model2'
            }
    
    def predict(self, image_path):
        """Run parallel prediction on both models"""
        # Load image
        pil_image = Image.open(image_path)
        
        # Run model1 (TensorFlow)
        try:
            tf_input = self.preprocess_for_model1(pil_image)
            features = self.feature_extractor.predict(tf_input)
            model1_probs = self.tf_model.predict(features)[0]
            model1_class_idx = np.argmax(model1_probs)
            model1_confidence = model1_probs[model1_class_idx]
            model1_class = self.class_names_model1[model1_class_idx]
        except Exception as e:
            print("TensorFlow prediction error:", e)
            model1_class = "Error"
            model1_confidence = 0.0
            model1_probs = []
        
        # Run model2 (PyTorch)
        try:
            pt_input = self.preprocess_for_model2(pil_image)
            with torch.no_grad():
                model2_output = self.torch_model(pt_input)
            model2_probs = F.softmax(model2_output, dim=-1)[0]
            model2_class_idx = model2_probs.argmax(-1).item()
            model2_confidence = model2_probs[model2_class_idx].item()
            model2_class = self.class_names_model2[model2_class_idx]
        except Exception as e:
            print("PyTorch prediction error:", e)
            model2_class = "Error"
            model2_confidence = 0.0
            model2_probs = []
        
        return {
            'model1': {
                'class': model1_class,
                'confidence': float(model1_confidence),
                'all_probs': model1_probs.tolist() if len(model1_probs) > 0 else []
            },
            'model2': {
                'class': model2_class,
                'confidence': model2_confidence,
                'all_probs': model2_probs.tolist() if torch.is_tensor(model2_probs) else []
            },
            'combined': self.combine_results(
                model1_class, model1_confidence,
                model2_class, model2_confidence
            )
        }

    def save(self, path):
        """Custom save method that handles both models"""
        # Create a temporary directory
        temp_dir = Path(path).with_suffix('.tmp')
        temp_dir.mkdir(exist_ok=True)
        
        # Save TensorFlow model with proper extension
        tf_model_path = temp_dir / 'tf_model.keras'  # Using recommended .keras extension
        self.tf_model.save(tf_model_path)
        
        # Save PyTorch model
        torch_model_path = temp_dir / 'torch_model.pt'
        torch.jit.save(self.torch_model, torch_model_path)
        
        # Save the wrapper's state (excluding the models)
        state = {
            'tf_model_path': str(tf_model_path.absolute()),
            'torch_model_path': str(torch_model_path.absolute()),
            'class_names1': self.class_names_model1,
            'class_names2': self.class_names_model2,
            'conflict_rules': self.conflict_rules
        }
        
        with open(path, 'wb') as f:
            dill.dump(state, f)

    @classmethod
    def load(cls, path):
        """Custom load method"""
        with open(path, 'rb') as f:
            state = dill.load(f)
        
        # For TensorFlow model, use either:
        # Option 1: If saved as .keras
        tf_model = tf.keras.models.load_model(state['tf_model_path'])
        
        # Option 2: If you need to support older .h5 format
        # tf_model = tf.keras.models.load_model(state['tf_model_path'], compile=False)
        
        return cls(
            tf_model_path=state['tf_model_path'],
            torch_model_path=state['torch_model_path'],
            class_names1=state['class_names1'],
            class_names2=state['class_names2'],
            conflict_rules=state['conflict_rules']
        )

# ====== CONFIGURATION ======
CLASS_NAMES_MODEL1 = [
    'Eczema', 'Melanoma', 'Atopic Dermatitis', 
    'Basal Cell Carcinoma', 'Melanocytic Nevi', 'Benign Keratosis'
]

CLASS_NAMES_MODEL2 = [
    'Basal Cell Carcinoma', 'Darier_s Disease', 'Epidermolysis Bullosa Pruriginosa',
    'Hailey-Hailey Disease', 'Herpes Simplex', 'Impetigo', 'Larva Migrans',
    'Leprosy Borderline', 'Leprosy Lepromatous', 'Leprosy Tuberculoid',
    'Lichen Planus', 'Lupus Erythematosus Chronicus Discoides', 'Melanoma',
    'Molluscum Contagiosum', 'Mycosis Fungoides', 'Neurofibromatosis',
    'Papilomatosis Confluentes And Reticulate', 'Pediculosis Capitis',
    'Pityriasis Rosea', 'Porokeratosis Actinic', 'Psoriasis', 'Tinea Corporis',
    'Tinea Nigra', 'Tungiasis', 'actinic keratosis', 'dermatofibroma', 'nevus',
    'pigmented benign keratosis', 'seborrheic keratosis', 'squamous cell carcinoma',
    'vascular lesion'
]

CONFLICT_RULES = {
    ('Eczema', 'Psoriasis'): 'Psoriasis',
    ('Benign Keratosis', 'actinic keratosis'): 'actinic keratosis',
    ('Melanocytic Nevi', 'Melanoma'): 'Melanoma',
    ('Atopic Dermatitis', 'Eczema'): 'Eczema'
}


2025-05-03 22:29:08.002323: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746300548.065555   10307 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746300548.089911   10307 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746300548.234231   10307 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746300548.234294   10307 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746300548.234297   10307 computation_placer.cc:177] computation placer alr

In [86]:
# ====== INITIALIZE AND SAVE ======
combined_model = CombinedSkinModel(
    tf_model_path='final_model.h5',
    torch_model_path='model_mobile_quantized.pt',
    class_names1=CLASS_NAMES_MODEL1,
    class_names2=CLASS_NAMES_MODEL2,
    conflict_rules=CONFLICT_RULES
)

In [87]:
# Save the model
combined_model.save('combined_skin_model.dill')

In [3]:
# Load and test
loaded_model = CombinedSkinModel.load('combined_skin_model.dill')
result = loaded_model.predict('/home/rabieash/projects/GP/Psoriasis_copy.original.width-320.jpg')
print(result)

I0000 00:00:1746300594.491221   10307 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4057 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5
I0000 00:00:1746300598.173183   10396 service.cc:152] XLA service 0x7efda0003380 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746300598.173315   10396 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 2060, Compute Capability 7.5
2025-05-03 22:29:58.276533: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
E0000 00:00:1746300598.884928   10396 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If buildi

TensorFlow prediction error: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/home/rabieash/miniconda3/envs/pytorch_env/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/home/rabieash/miniconda3/envs/pytorch_env/lib/python3.9/runpy.py", line 87, in _run_code

  File "/home/rabieash/miniconda3/envs/pytorch_env/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/rabieash/miniconda3/envs/pytorch_env/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/rabieash/miniconda3/envs/pytorch_env/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/rabieash/miniconda3/envs/pytorch_env/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/rabieash/miniconda3/envs/pytorch_env/lib/python3.9/asyncio/base_events.py", line 601, in run_forever

  File "/home/rabieash/

In [4]:
# Display results
print("\n=== Prediction Results ===")
print("ResNet50 Prediction:", result['model1']['class'], f"({result['model1']['confidence']:.1%})")
print("DINOv2 Prediction:", result['model2']['class'], f"({result['model2']['confidence']:.1%})")
print("Final Decision:", result['combined']['final_class'], 
      f"({result['combined']['confidence']:.1%})",
      f"[Source: {result['combined']['source']}]")


=== Prediction Results ===
ResNet50 Prediction: Error (0.0%)
DINOv2 Prediction: Psoriasis (98.6%)
Final Decision: Psoriasis (98.6%) [Source: model2]


In [3]:
import os
import dill
import torch
import torch.nn.functional as F
import tensorflow as tf
import numpy as np
from PIL import Image
from torchvision import transforms
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from pathlib import Path

class CombinedSkinModel:
    def __init__(self, tf_model=None, torch_model=None, class_names1=None, class_names2=None, conflict_rules=None):
        self._tf_model = tf_model
        self._torch_model = torch_model
        self.class_names_model1 = class_names1 or []
        self.class_names_model2 = class_names2 or []
        self.conflict_rules = conflict_rules or {}
        self._feature_extractor = None

    @property
    def tf_model(self):
        if self._tf_model is None:
            raise ValueError("TensorFlow model not loaded")
        return self._tf_model

    @property
    def torch_model(self):
        if self._torch_model is None:
            raise ValueError("PyTorch model not loaded")
        return self._torch_model

    @property
    def feature_extractor(self):
        if self._feature_extractor is None:
            self._feature_extractor = tf.keras.applications.ResNet50(
                weights='imagenet', 
                include_top=False, 
                pooling='avg'
            )
        return self._feature_extractor

    def preprocess_for_model1(self, image):
        """Preprocess image for TensorFlow ResNet50 model"""
        image = image.resize((224, 224))
        image_array = img_to_array(image)
        image_array = np.expand_dims(image_array, axis=0)
        return preprocess_input(image_array)

    def preprocess_for_model2(self, image):
        """Preprocess image for PyTorch DINOv2 model"""
        preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        return preprocess(image.convert("RGB")).unsqueeze(0)
    
    def combine_results(self, class1, conf1, class2, conf2):
        """Class-aware combination logic"""
        # Check both orderings of class pairs
        if (class1, class2) in self.conflict_rules:
            correct_class = self.conflict_rules[(class1, class2)]
            return {
                'final_class': correct_class, 
                'confidence': max(conf1, conf2),
                'source': 'expert_rules'
            }
        elif (class2, class1) in self.conflict_rules:
            correct_class = self.conflict_rules[(class2, class1)]
            return {
                'final_class': correct_class,
                'confidence': max(conf1, conf2),
                'source': 'expert_rules'
            }
        
        # Default confidence-based approach with preference to model2 (DINOv2)
        if conf1 > conf2 + 0.15:  # If model1 is significantly more confident
            return {
                'final_class': class1,
                'confidence': conf1,
                'source': 'model1'
            }
        else:  # Default to model2's prediction
            return {
                'final_class': class2,
                'confidence': conf2,
                'source': 'model2'
            }
    
    def predict(self, image_path):
        """Run parallel prediction on both models"""
        # Load image
        pil_image = Image.open(image_path)
        
        # Run model1 (TensorFlow)
        try:
            tf_input = self.preprocess_for_model1(pil_image)
            features = self.feature_extractor.predict(tf_input)
            model1_probs = self.tf_model.predict(features)[0]
            model1_class_idx = np.argmax(model1_probs)
            model1_confidence = model1_probs[model1_class_idx]
            model1_class = self.class_names_model1[model1_class_idx]
        except Exception as e:
            print("TensorFlow prediction error:", e)
            model1_class = "Error"
            model1_confidence = 0.0
            model1_probs = []
        
        # Run model2 (PyTorch)
        try:
            pt_input = self.preprocess_for_model2(pil_image)
            with torch.no_grad():
                model2_output = self.torch_model(pt_input)
            model2_probs = F.softmax(model2_output, dim=-1)[0]
            model2_class_idx = model2_probs.argmax(-1).item()
            model2_confidence = model2_probs[model2_class_idx].item()
            model2_class = self.class_names_model2[model2_class_idx]
        except Exception as e:
            print("PyTorch prediction error:", e)
            model2_class = "Error"
            model2_confidence = 0.0
            model2_probs = []
        
        return {
            'model1': {
                'class': model1_class,
                'confidence': float(model1_confidence),
                'all_probs': model1_probs.tolist() if len(model1_probs) > 0 else []
            },
            'model2': {
                'class': model2_class,
                'confidence': model2_confidence,
                'all_probs': model2_probs.tolist() if torch.is_tensor(model2_probs) else []
            },
            'combined': self.combine_results(
                model1_class, model1_confidence,
                model2_class, model2_confidence
            )
        }

    def save(self, path):
        """Save the complete model with all weights in a single file"""
        # Convert models to bytes
        tf_model_bytes = None
        if self._tf_model is not None:
            with open("temp_tf_model.keras", "wb") as f:
                self._tf_model.save(f.name)
            with open("temp_tf_model.keras", "rb") as f:
                tf_model_bytes = f.read()
            os.remove("temp_tf_model.keras")

        torch_model_bytes = None
        if self._torch_model is not None:
            torch.jit.save(self._torch_model, "temp_torch_model.pt")
            with open("temp_torch_model.pt", "rb") as f:
                torch_model_bytes = f.read()
            os.remove("temp_torch_model.pt")

        # Save everything in one file
        state = {
            'tf_model_bytes': tf_model_bytes,
            'torch_model_bytes': torch_model_bytes,
            'class_names1': self.class_names_model1,
            'class_names2': self.class_names_model2,
            'conflict_rules': self.conflict_rules
        }
        
        with open(path, 'wb') as f:
            dill.dump(state, f)

    @classmethod
    def load(cls, path):
        """Load the complete model from a single file"""
        with open(path, 'rb') as f:
            state = dill.load(f)
        
        # Reconstruct models from bytes
        tf_model = None
        if state['tf_model_bytes'] is not None:
            with open("temp_reload_tf.keras", "wb") as f:
                f.write(state['tf_model_bytes'])
            tf_model = tf.keras.models.load_model("temp_reload_tf.keras")
            os.remove("temp_reload_tf.keras")

        torch_model = None
        if state['torch_model_bytes'] is not None:
            with open("temp_reload_torch.pt", "wb") as f:
                f.write(state['torch_model_bytes'])
            torch_model = torch.jit.load("temp_reload_torch.pt")
            torch_model.eval()
            os.remove("temp_reload_torch.pt")

        return cls(
            tf_model=tf_model,
            torch_model=torch_model,
            class_names1=state['class_names1'],
            class_names2=state['class_names2'],
            conflict_rules=state['conflict_rules']
        )

2025-05-03 22:44:52.781359: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746301492.804734   10716 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746301492.811756   10716 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746301492.839420   10716 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746301492.839442   10716 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746301492.839444   10716 computation_placer.cc:177] computation placer alr

In [6]:
# First-time setup (only need to do this once)
if not os.path.exists("combined_model.dill"):
        print("Creating combined model...")
        # Load original models
        tf_model = tf.keras.models.load_model('final_model.h5')
        torch_model = torch.jit.load('model_mobile_quantized.pt')
        
        # Define your class names and rules
        CLASS_NAMES_MODEL1 = [
            'Eczema', 'Melanoma', 'Atopic Dermatitis', 
            'Basal Cell Carcinoma', 'Melanocytic Nevi', 'Benign Keratosis'
        ]
        
        CLASS_NAMES_MODEL2 = [
            'Basal Cell Carcinoma', 'Darier_s Disease', 'Epidermolysis Bullosa Pruriginosa',
            'Hailey-Hailey Disease', 'Herpes Simplex', 'Impetigo', 'Larva Migrans',
            'Leprosy Borderline', 'Leprosy Lepromatous', 'Leprosy Tuberculoid',
            'Lichen Planus', 'Lupus Erythematosus Chronicus Discoides', 'Melanoma',
            'Molluscum Contagiosum', 'Mycosis Fungoides', 'Neurofibromatosis',
            'Papilomatosis Confluentes And Reticulate', 'Pediculosis Capitis',
            'Pityriasis Rosea', 'Porokeratosis Actinic', 'Psoriasis', 'Tinea Corporis',
            'Tinea Nigra', 'Tungiasis', 'actinic keratosis', 'dermatofibroma', 'nevus',
            'pigmented benign keratosis', 'seborrheic keratosis', 'squamous cell carcinoma',
            'vascular lesion'
        ]
        
        CONFLICT_RULES = {
            ('Eczema', 'Psoriasis'): 'Psoriasis',
            ('Benign Keratosis', 'actinic keratosis'): 'actinic keratosis',
            ('Melanocytic Nevi', 'Melanoma'): 'Melanoma',
            ('Atopic Dermatitis', 'Eczema'): 'Eczema'
        }

/home/rabieash/miniconda3/envs/pytorch_env/lib/python3.9/site-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Creating combined model...


In [7]:
# Create and save combined model
combined_model = CombinedSkinModel(
            tf_model=tf_model,
            torch_model=torch_model,
            class_names1=CLASS_NAMES_MODEL1,
            class_names2=CLASS_NAMES_MODEL2,
            conflict_rules=CONFLICT_RULES
        )
combined_model.save('combined_model.dill')
print("Saved combined model to combined_model.dill")

Saved combined model to combined_model.dill


In [4]:
# Normal usage (just load the combined model)
print("Loading combined model...")
model = CombinedSkinModel.load('combined_model.dill')
        
# Make predictions
result = model.predict('/home/rabieash/projects/GP/Psoriasis_copy.original.width-320.jpg')
print("\nPrediction Results:")
print(f"ResNet50: {result['model1']['class']} ({result['model1']['confidence']:.1%})")
print(f"DINOv2: {result['model2']['class']} ({result['model2']['confidence']:.1%})")
print(f"Final: {result['combined']['final_class']} ({result['combined']['confidence']:.1%}) [Source: {result['combined']['source']}]")

Loading combined model...


2025-05-03 22:45:08.679437: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step

I0000 00:00:1746301512.117313   10820 service.cc:152] XLA service 0x7f4e9c2640f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746301512.117422   10820 service.cc:160]   StreamExecutor device (0): Host, Default Version
2025-05-03 22:45:12.125348: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1746301512.302233   10820 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step

Prediction Results:
ResNet50: Eczema (99.9%)
DINOv2: Psoriasis (98.6%)
Final: Psoriasis (99.9%) [Source: expert_rules]
